In [ ]:
!pip install plotly
!pip install lightgbm

In [ ]:
import mlrun
# get/create a project and register the data prep and trainer function in it
project = mlrun.get_or_create_project(
    name="test-serv-with-remote", user_project=False, context="./"
)


In [ ]:
func = project.set_function(name='data-prep', func='src/data_prep.py', image='mlrun/mlrun',
                     handler='data_preparation', kind="job", requirements=["lightgbm"])
func.deploy()
func = project.set_function(name='trainer_lgbm', func='src/trainer_lgbm.py', image='mlrun/mlrun',
                     handler='train', kind="job", requirements=["lightgbm"])
func.deploy()
project.save()

<a id="prepare-data"></a>
## Prepare your data for training and testing

**Run the data prepartion function (locally) using the dataset from 'data/dataset.csv':**

In [ ]:
# execute the function through MLRun SDK (can run locally or over the k8s cluster)
data_prep = project.run_function("data-prep", inputs={"dataset": "https://s3.us-east-1.wasabisys.com/iguazio/data/nyc-taxi/train.csv"})

# # get the returned data artifact
# train_dataset = data_prep.artifact("train_dataset").as_df()
# train_dataset.head()

In [ ]:
inputs_for_trainer = data_prep.outputs["train_dataset"]

<a id="train-function"></a>
## Run the training function and log the artifacts and model


In [ ]:
# execute the function through MLRun SDK (can run locally or over the k8s cluster)
train = project.run_function("trainer_lgbm",
                             inputs={"train_set": inputs_for_trainer})

# print the results of the training run
# train.outputs

In [ ]:
# get the returned data artifact
train.artifact('feature-importance').show()